In [ ]:
import pickle
with open('/content/drive/MyDrive/Colab_Notebooks/MDDdataset/symptom_sum_top16/train.pkl', 'rb') as f:
      raw_train_data = pickle.load(f)
with open('/content/drive/MyDrive/Colab_Notebooks/MDDdataset/symptom_sum_top16/test.pkl', 'rb') as f:
      raw_test_data = pickle.load(f)
with open('/content/drive/MyDrive/Colab_Notebooks/MDDdataset/symptom_sum_top16/val.pkl', 'rb') as f:
      raw_val_data = pickle.load(f)

In [ ]:
import pandas as pd
train_data=pd.DataFrame(raw_train_data)
test_data=pd.DataFrame(raw_test_data)
val_data=pd.DataFrame(raw_val_data)

In [ ]:

df_all = pd.concat([train_data, val_data], axis=0)
df_all = pd.concat([df_all, test_data], axis=0)

In [ ]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, make_scorer
from sklearn.utils import resample
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler
from scipy.sparse import hstack, csr_matrix

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text_list):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    processed_text = []
    text = ' '.join(text_list) if isinstance(text_list, list) else text_list
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and len(word) > 1]
    return ' '.join(words)

def calculate_assessment_scores(text):
    keywords = {
        1: ['limit food', 'diet', 'eat less'],
        2: ['not eating', 'skipping meals', 'fasting'],
        3: ['obsessed with food', 'focus on calories'],
        4: ['worried about weight', 'shape concern'],
        5: ['fear of weight gain', 'avoid weight gain'],
        6: ['desire to lose weight', 'weight loss'],
        7: ['self-induced vomiting', 'use of laxatives'],
        8: ['excessive exercise', 'compulsive workout'],
        9: ['loss of control eating', 'binge eating'],
        10: ['large amount of food', 'overeating'],
        11: ['self-worth by weight', 'body image'],
        12: ['dissatisfied with weight', 'unhappy with body']
    }
    scores = np.zeros(len(keywords))
    for key, phrases in keywords.items():
        for phrase in phrases:
            if phrase in text:
                scores[key-1] = 1
    return scores

# Load your data here
combined_data = df_all
combined_data['processed_posts'] = combined_data['selected_posts'].apply(preprocess_text)

# Setup TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, min_df=1)
X_text = vectorizer.fit_transform(combined_data['processed_posts'])

# Calculate and prepare assessment features
combined_data['assessment_scores'] = combined_data['processed_posts'].apply(calculate_assessment_scores)
assessment_features = np.vstack(combined_data['assessment_scores'])
assessment_features_csr = csr_matrix(assessment_features)  # Convert to CSR format

# Combine text features with assessment scores
X_combined = hstack([X_text, assessment_features_csr]).tocsr()  # Ensure it's in CSR format

y = combined_data['diseases'].apply(lambda x: 1 if 'eating' in x else 0).values

# Define the logistic regression model with oversampling
model = LogisticRegression(max_iter=1000)
pipeline = make_pipeline(RandomOverSampler(random_state=42), model)

from sklearn.metrics import confusion_matrix, f1_score, make_scorer
# Metrics setup
scoring = {
    'f1': 'f1',
    'sensitivity': make_scorer(lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[1, 1] / (confusion_matrix(y_true, y_pred)[1, 1] + confusion_matrix(y_true, y_pred)[1, 0])),
    'specificity': make_scorer(lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[0, 0] / (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1]))
}

# Bootstrapping loop
n_bootstraps = 100
f1_scores, sensitivities, specificities = [], [], []
for _ in range(n_bootstraps):
    indices = resample(np.arange(len(y)), replace=True)
    X_sample, y_sample = X_combined[indices], y[indices]

    pipeline.fit(X_sample, y_sample)
    y_pred = pipeline.predict(X_sample)

    tn, fp, fn, tp = confusion_matrix(y_sample, y_pred).ravel()
    f1_scores.append(f1_score(y_sample, y_pred))
    sensitivities.append(tp / (tp + fn))
    specificities.append(tn / (tn + fp))

# Calculate mean and standard deviation for metrics
print("Bootstrap Results:")
print(f"F1 Score: Mean = {np.mean(f1_scores):.4f}, Std = {np.std(f1_scores):.4f}")
print(f"Sensitivity: Mean = {np.mean(sensitivities):.4f}, Std = {np.std(sensitivities):.4f}")
print(f"Specificity: Mean = {np.mean(specificities):.4f}, Std = {np.std(specificities):.4f}")


Bootstrap Results:
F1 Score: Mean = 0.5332, Std = 0.0235
Sensitivity: Mean = 1.0000, Std = 0.0000
Specificity: Mean = 0.9907, Std = 0.0010


In [ ]:
import numpy as np
import pandas as pd
import ast
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, make_scorer, f1_score
from sklearn.model_selection import StratifiedKFold, cross_validate
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler
from scipy.sparse import hstack

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text_list):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    processed_text = []
    text = ' '.join(text_list)
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and len(word) > 1]
    return ' '.join(words)

def calculate_assessment_scores(text):
    keywords = {
        1: ['limit food', 'diet', 'eat less'],
        2: ['not eating', 'skipping meals', 'fasting'],
        3: ['obsessed with food', 'focus on calories'],
        4: ['worried about weight', 'shape concern'],
        5: ['fear of weight gain', 'avoid weight gain'],
        6: ['desire to lose weight', 'weight loss'],
        7: ['self-induced vomiting', 'use of laxatives'],
        8: ['excessive exercise', 'compulsive workout'],
        9: ['loss of control eating', 'binge eating'],
        10: ['large amount of food', 'overeating'],
        11: ['self-worth by weight', 'body image'],
        12: ['dissatisfied with weight', 'unhappy with body']
    }

    scores = {key: 0 for key in keywords}
    for key, phrases in keywords.items():
        for phrase in phrases:
            if phrase in text:
                scores[key] = 1  # You can modify the scoring logic as needed
    return scores

combined_data = df_all

# Preprocess and calculate scores
combined_data['processed_posts'] = combined_data['selected_posts'].apply(preprocess_text)
combined_data['assessment_scores'] = combined_data['processed_posts'].apply(calculate_assessment_scores)

# Extract features from assessment scores
for key in combined_data['assessment_scores'].iloc[0].keys():
    combined_data[key] = combined_data['assessment_scores'].apply(lambda x: x[key])

# Setup TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000, min_df=1)
X_text = vectorizer.fit_transform(combined_data['processed_posts'])

# Combine text features with assessment scores
assessment_features = np.array(combined_data[list(combined_data['assessment_scores'].iloc[0].keys())])
X_combined = hstack([X_text, assessment_features])

y = combined_data['diseases'].apply(lambda x: 1 if 'eating' in x else 0)

# Define the logistic regression model with oversampling
model = LogisticRegression(max_iter=1000)
pipeline = make_pipeline(RandomOverSampler(random_state=42), model)

# Metrics and cross-validation
scoring = {'f1': 'f1', 'sensitivity': make_scorer(lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[1, 1] / (confusion_matrix(y_true, y_pred)[1, 1] + confusion_matrix(y_true, y_pred)[1, 0])),
           'specificity': make_scorer(lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[0, 0] / (confusion_matrix(y_true, y_pred)[0, 0] + confusion_matrix(y_true, y_pred)[0, 1]))}

cv = StratifiedKFold(n_splits=5)
results = cross_validate(pipeline, X_combined, y, cv=cv, scoring=scoring)

# Display results
print("Cross-Validation Results:")
for metric in ['f1', 'sensitivity', 'specificity']:
    mean_score = np.mean(results[f'test_{metric}'])
    std_score = np.std(results[f'test_{metric}'])
    print(f"{metric.capitalize()}: Mean = {mean_score:.4f}, Std = {std_score:.4f}")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Cross-Validation Results:
F1: Mean = 0.3306, Std = 0.2112
Sensitivity: Mean = 0.4058, Std = 0.0569
Specificity: Mean = 0.9562, Std = 0.0766
